In [3]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

<IPython.core.display.Javascript object>

In [4]:
project_name = '02-insurance-linear-regression'

In [5]:
DATASET_UTL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = 'insurance.csv'
download_url(DATASET_UTL, '.')

Using downloaded and verified file: ./insurance.csv


In [6]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [7]:
your_name = "presz"

In [8]:
def customized_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state = int(ord(rand_str[0])))
    
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    
    if(ord(rand_str[3]) % 2 ==1):
        dataframe = dataframe.drop(['region'], axis = 1)
    return dataframe

In [9]:
dataframe = customized_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
13,56,female,45.3948,0,no,11201.624978
367,42,female,28.4829,2,no,8097.231762
623,18,male,38.2299,0,yes,34964.019056
1281,47,female,31.5153,2,yes,24781.055536
966,51,male,28.2663,2,yes,24207.056881


In [10]:
#Q1
num_rows = (len(dataframe))
print(num_rows)

1271


In [11]:
#Q2
num_cols = len(dataframe.columns)
print(num_cols)

6


In [12]:
#Q3
input_cols = [cols for cols in dataframe.columns]
input_cols = input_cols[0:5]
print(input_cols)

['age', 'sex', 'bmi', 'children', 'smoker']


In [13]:
#Q4 
categorical_cols = dataframe.select_dtypes(exclude = ['int64', 'float64'])
print(categorical_cols)

         sex smoker
13    female     no
367   female     no
623     male    yes
1281  female    yes
966     male    yes
...      ...    ...
1148    male     no
628     male     no
546     male     no
971   female     no
932     male     no

[1271 rows x 2 columns]


In [14]:
#Q5
# dataframe.info()
output_cols = ['charges']
print(output_cols)

['charges']


In [15]:
## jovian.commit

In [16]:
def dataframe_to_arrays(dataframe):
    dataframe1 = dataframe.copy(deep=True)
    
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
        
    inputs_array=dataframe1[input_cols].to_numpy()
    targets_array=dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [17]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[56.    ,  0.    , 45.3948,  0.    ,  0.    ],
        [42.    ,  0.    , 28.4829,  2.    ,  0.    ],
        [18.    ,  1.    , 38.2299,  0.    ,  1.    ],
        ...,
        [28.    ,  1.    , 40.3959,  0.    ,  0.    ],
        [34.    ,  0.    , 26.8584,  0.    ,  0.    ],
        [46.    ,  1.    , 29.412 ,  5.    ,  0.    ]]),
 array([[11201.624978 ],
        [ 8097.2317615],
        [34964.0190565],
        ...,
        [ 3301.5351165],
        [ 5042.300164 ],
        [10197.9397   ]]))

In [106]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).float()

## Using .float to cast it to 32 bit float

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [19]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

In [114]:
mean = 0
for yb in targets:
    mean +=yb.item()
print(mean/len(targets))

13458.252626480595


In [20]:
dataset = TensorDataset(inputs, targets) ## 1271

In [21]:
val_percent = 0.121
val_size = int(num_rows * val_percent) ## 153
train_size = num_rows - val_size ## 1118

train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [193]:
batch_size = 26

In [194]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [195]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[53.0000,  1.0000, 35.7390,  0.0000,  0.0000],
        [59.0000,  0.0000, 39.6720,  2.0000,  0.0000],
        [27.0000,  1.0000, 34.7700,  0.0000,  0.0000],
        [33.0000,  0.0000, 48.9516,  3.0000,  0.0000],
        [48.0000,  0.0000, 31.8402,  4.0000,  0.0000],
        [37.0000,  0.0000, 54.2640,  2.0000,  1.0000],
        [52.0000,  0.0000, 35.0892,  1.0000,  0.0000],
        [27.0000,  1.0000, 33.2310,  0.0000,  1.0000],
        [51.0000,  1.0000, 27.8331,  4.0000,  0.0000],
        [40.0000,  1.0000, 34.0860,  2.0000,  0.0000],
        [33.0000,  1.0000, 33.5160,  4.0000,  0.0000],
        [50.0000,  1.0000, 51.0378,  1.0000,  0.0000],
        [35.0000,  1.0000, 31.6236,  2.0000,  1.0000],
        [48.0000,  0.0000, 47.0022,  4.0000,  0.0000],
        [52.0000,  0.0000, 50.9580,  3.0000,  0.0000],
        [39.0000,  0.0000, 38.8740,  3.0000,  0.0000],
        [64.0000,  0.0000, 34.3311,  3.0000,  0.0000],
        [18.0000,  0.0000, 36.6168,  2.0000,  0.0000],
  

In [25]:
jovian.commit(environment = None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "presz/assignment-02" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/presz/assignment-02


'https://jovian.ai/presz/assignment-02'

## Creating a Linear Regression Model

In [196]:
input_size = len(input_cols)
output_size = len(output_cols)
print(input_size, output_size)

5 1


In [202]:
import torch.nn.functional as F
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        
    def forward(self, xb):
        out = self.linear(xb)
        return out

    def training_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.huber_loss(out, targets, delta=0.007) #############3
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.huber_loss(out, targets, delta=0.007) #####################
        return {'val_loss': loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        if(epoch+1)%20 ==0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [229]:
test_model = InsuranceModel()
list(test_model.parameters())

[Parameter containing:
 tensor([[ 0.0118,  0.4348,  0.1691, -0.1694, -0.3209]], requires_grad=True),
 Parameter containing:
 tensor([0.1014], requires_grad=True)]

In [199]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history=[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [230]:
result = evaluate(test_model, val_loader)
print(result)

{'val_loss': 99.48657989501953}


In [231]:
epochs = 100
lr = 0.5
history1 = fit(epochs, lr, test_model, train_loader, val_loader)
list(test_model.parameters())

Epoch [20], val_loss: 64.2684
Epoch [40], val_loss: 61.9638
Epoch [60], val_loss: 61.7693
Epoch [80], val_loss: 61.4970
Epoch [100], val_loss: 61.2046


[Parameter containing:
 tensor([[162.8654,   1.1501,  60.3865,   3.2670,   2.6810]],
        requires_grad=True),
 Parameter containing:
 tensor([1.9127], requires_grad=True)]

In [232]:
epochs = 1000
lr = 0.05
history2=fit(epochs, lr, test_model, train_loader, val_loader)

Epoch [20], val_loss: 61.1745
Epoch [40], val_loss: 61.1452
Epoch [60], val_loss: 61.1165
Epoch [80], val_loss: 61.0883
Epoch [100], val_loss: 61.0599
Epoch [120], val_loss: 61.0310
Epoch [140], val_loss: 61.0021
Epoch [160], val_loss: 60.9734
Epoch [180], val_loss: 60.9450
Epoch [200], val_loss: 60.9163
Epoch [220], val_loss: 60.8876
Epoch [240], val_loss: 60.8588
Epoch [260], val_loss: 60.8300
Epoch [280], val_loss: 60.8017
Epoch [300], val_loss: 60.7732
Epoch [320], val_loss: 60.7440
Epoch [340], val_loss: 60.7150
Epoch [360], val_loss: 60.6875
Epoch [380], val_loss: 60.6608
Epoch [400], val_loss: 60.6337
Epoch [420], val_loss: 60.6071
Epoch [440], val_loss: 60.5812
Epoch [460], val_loss: 60.5553
Epoch [480], val_loss: 60.5294
Epoch [500], val_loss: 60.5036
Epoch [520], val_loss: 60.4777
Epoch [540], val_loss: 60.4523
Epoch [560], val_loss: 60.4271
Epoch [580], val_loss: 60.4017
Epoch [600], val_loss: 60.3774
Epoch [620], val_loss: 60.3531
Epoch [640], val_loss: 60.3288
Epoch [660],

In [233]:
epochs = 500
lr = 0.1
history3=fit(epochs, lr, test_model, train_loader, val_loader)

Epoch [20], val_loss: 59.8434
Epoch [40], val_loss: 59.7949
Epoch [60], val_loss: 59.7484
Epoch [80], val_loss: 59.7021
Epoch [100], val_loss: 59.6572
Epoch [120], val_loss: 59.6128
Epoch [140], val_loss: 59.5670
Epoch [160], val_loss: 59.5201
Epoch [180], val_loss: 59.4734
Epoch [200], val_loss: 59.4268
Epoch [220], val_loss: 59.3817
Epoch [240], val_loss: 59.3389
Epoch [260], val_loss: 59.3003
Epoch [280], val_loss: 59.2619
Epoch [300], val_loss: 59.2235
Epoch [320], val_loss: 59.1864
Epoch [340], val_loss: 59.1515
Epoch [360], val_loss: 59.1166
Epoch [380], val_loss: 59.0823
Epoch [400], val_loss: 59.0476
Epoch [420], val_loss: 59.0156
Epoch [440], val_loss: 58.9844
Epoch [460], val_loss: 58.9528
Epoch [480], val_loss: 58.9201
Epoch [500], val_loss: 58.8866


In [234]:
epochs = 1000
lr = 0.2
history4=fit(epochs, lr, test_model, train_loader, val_loader)

Epoch [20], val_loss: 58.8298
Epoch [40], val_loss: 58.7725
Epoch [60], val_loss: 58.7173
Epoch [80], val_loss: 58.6679
Epoch [100], val_loss: 58.6293
Epoch [120], val_loss: 58.5924
Epoch [140], val_loss: 58.5589
Epoch [160], val_loss: 58.5299
Epoch [180], val_loss: 58.5016
Epoch [200], val_loss: 58.4715
Epoch [220], val_loss: 58.4438
Epoch [240], val_loss: 58.4192
Epoch [260], val_loss: 58.3955
Epoch [280], val_loss: 58.3762
Epoch [300], val_loss: 58.3569
Epoch [320], val_loss: 58.3406
Epoch [340], val_loss: 58.3225
Epoch [360], val_loss: 58.3032
Epoch [380], val_loss: 58.2888
Epoch [400], val_loss: 58.2724
Epoch [420], val_loss: 58.2544
Epoch [440], val_loss: 58.2400
Epoch [460], val_loss: 58.2343
Epoch [480], val_loss: 58.2308
Epoch [500], val_loss: 58.2266
Epoch [520], val_loss: 58.2166
Epoch [540], val_loss: 58.2074
Epoch [560], val_loss: 58.2011
Epoch [580], val_loss: 58.1944
Epoch [600], val_loss: 58.1899
Epoch [620], val_loss: 58.1856
Epoch [640], val_loss: 58.1816
Epoch [660],

In [235]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "presz/assignment-02" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/presz/assignment-02


'https://jovian.ai/presz/assignment-02'

In [239]:
epochs = 1000
lr = 0.01
history5 = fit(epochs, lr, test_model, train_loader, val_loader)

Epoch [20], val_loss: 58.1219
Epoch [40], val_loss: 58.1218
Epoch [60], val_loss: 58.1218
Epoch [80], val_loss: 58.1217
Epoch [100], val_loss: 58.1216
Epoch [120], val_loss: 58.1215
Epoch [140], val_loss: 58.1214
Epoch [160], val_loss: 58.1213
Epoch [180], val_loss: 58.1212
Epoch [200], val_loss: 58.1211
Epoch [220], val_loss: 58.1210
Epoch [240], val_loss: 58.1209
Epoch [260], val_loss: 58.1208
Epoch [280], val_loss: 58.1207
Epoch [300], val_loss: 58.1206
Epoch [320], val_loss: 58.1205
Epoch [340], val_loss: 58.1205
Epoch [360], val_loss: 58.1204
Epoch [380], val_loss: 58.1203
Epoch [400], val_loss: 58.1201
Epoch [420], val_loss: 58.1201
Epoch [440], val_loss: 58.1200
Epoch [460], val_loss: 58.1199
Epoch [480], val_loss: 58.1198
Epoch [500], val_loss: 58.1197
Epoch [520], val_loss: 58.1196
Epoch [540], val_loss: 58.1195
Epoch [560], val_loss: 58.1194
Epoch [580], val_loss: 58.1193
Epoch [600], val_loss: 58.1192
Epoch [620], val_loss: 58.1191
Epoch [640], val_loss: 58.1190
Epoch [660],

In [241]:
val_loss = evaluate(test_model, val_loader)
print(val_loss)

{'val_loss': 58.11732864379883}


In [243]:
jovian.log_metrics(val_loss = val_loss)

[jovian] Metrics logged.


In [255]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(input)
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target.item())
    print("Prediction:", prediction.item())

In [256]:
input, target = val_ds[0]
predict_single(input, target, test_model)

Input: tensor([22.0000,  0.0000, 26.4252,  0.0000,  0.0000])
Target: 2759.231201171875
Prediction: 3793.75341796875


In [257]:
input, target = val_ds[10]
predict_single(input, target, test_model)

Input: tensor([31.0000,  1.0000, 32.5983,  1.0000,  0.0000])
Target: 4286.02587890625
Prediction: 5688.2548828125


In [258]:
input, target = val_ds[26]
predict_single(input, target, test_model)

Input: tensor([41.0000,  0.0000, 36.0240,  0.0000,  0.0000])
Target: 6247.98828125
Prediction: 8003.9453125


In [252]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "presz/assignment-02" on https://jovian.ai/
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/presz/assignment-02


'https://jovian.ai/presz/assignment-02'

In [259]:
jovian.submit(assignment="zerotogans-a2")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "presz/assignment-02" on https://jovian.ai/
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/presz/assignment-02
[jovian] Submitting assignment..
[jovian] Verify your submission at https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model


In [264]:
def predict(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(input)
    prediction = predictions[0].detach()
    return(target.item()-prediction.item())

In [266]:
sum = 0
for i in range(batch_size):
    input, target = val_ds[i]
    sum += predict(input, target, test_model)
print(sum/batch_size)

4533.4151611328125
